<a href="https://colab.research.google.com/github/ridwaanhall/TOPSIS-Top-Ranked-Employee/blob/main/TOPSIS_Top_Ranked_Employee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### created by ridwaanhall
menentukan karyawan terbaik menggunakan metode topsis.

## Import libraries

In [1]:
import numpy as np
from tabulate import tabulate

## Input Rows and Cols

In [2]:
rows = int(input("Employee Count  : "))  # Example: 10.
cols = int(input("Criterion Count : "))  # Example: 5.

Employee Count  : 10
Criterion Count : 5


## Random

### Random Weights (0 to 1)

In [3]:
random_weights = np.random.rand(cols)
normalized_weights = random_weights / np.sum(random_weights)
weights = np.round(normalized_weights, 9)
print("Weights:", weights)
print("Total weights:", np.round(np.sum(weights), 2))

Weights: [0.02425691 0.33518449 0.3486661  0.02162994 0.27026255]
Total weights: 1.0


### Random Alternatives
based on rows

In [4]:
alternatives = [f"Employee {i+1}" for i in range(rows)]
print("Alternatives:", alternatives)

Alternatives: ['Employee 1', 'Employee 2', 'Employee 3', 'Employee 4', 'Employee 5', 'Employee 6', 'Employee 7', 'Employee 8', 'Employee 9', 'Employee 10']


### Random Matrix
this matrix after fuzzification. from A to 1, from 90 to 0.9, etc.

In [5]:
matrix = np.random.rand(rows, cols)
scaled_matrix = 0.5 + 0.5 * matrix
matrix = np.round(scaled_matrix, 10)
print("Matrix:")
print(matrix)

Matrix:
[[0.58093118 0.78804027 0.93735959 0.68790217 0.50972879]
 [0.96482187 0.79254673 0.57712766 0.98483751 0.71555147]
 [0.59826731 0.8087033  0.84349948 0.52823212 0.6502594 ]
 [0.74244427 0.70037885 0.89236209 0.55813118 0.88601567]
 [0.62064528 0.96760837 0.52395389 0.73808527 0.69683574]
 [0.7969034  0.79507857 0.8126029  0.90487483 0.67630721]
 [0.52071708 0.85483207 0.77903939 0.88774869 0.930064  ]
 [0.81267973 0.83476962 0.86565762 0.83119776 0.74025689]
 [0.57514336 0.54223742 0.64324724 0.61780963 0.50345129]
 [0.50576766 0.77703195 0.80050761 0.70633371 0.61424115]]


## Class

### TOPSIS

In [6]:
class TOPSIS:
    def __init__(self, matrix, weights, alternatives):
        self.matrix = matrix
        self.weights = weights
        self.alternatives = alternatives
        self.norm_matrix = None
        self.weighted_norm_matrix = None
        self.positive_ideal = None
        self.negative_ideal = None
        self.positive_distances = None
        self.negative_distances = None
        self.relative_closeness = None

    # Data Tables: Create tabulate alternatives, matrix, and weights
    def create_tabulate_alternatives(self):
        print("Alternatives:")
        print(tabulate(np.column_stack((self.alternatives, self.matrix)), headers=['Alternative'] + [f'Criterion {i+1}' for i in range(self.matrix.shape[1])], floatfmt=".9f"))

    # Step 1: Normalize the decision matrix
    def normalize_matrix(self):
        self.norm_matrix = np.copy(self.matrix)
        for col in range(self.matrix.shape[1]):
            norm = np.sqrt(np.sum(self.matrix[:, col] ** 2))
            self.norm_matrix[:, col] = self.matrix[:, col] / norm
        print("\nNormalized Decision Matrix:")
        print(tabulate(np.column_stack((self.alternatives, self.norm_matrix)), headers=['Alternative'] + [f'Criterion {i+1}' for i in range(self.norm_matrix.shape[1])], floatfmt=".9f"))

    # Step 2: Apply weights to the normalized matrix
    def weighted_normalized_matrix(self):
        self.weighted_norm_matrix = self.norm_matrix * self.weights
        print("\nWeighted Normalized Matrix:")
        print(tabulate(np.column_stack((self.alternatives, self.weighted_norm_matrix)), headers=['Alternative'] + [f'Criterion {i+1}' for i in range(self.weighted_norm_matrix.shape[1])], floatfmt=".9f"))

    # Step 3: Calculate positive and negative ideal solutions
    def calculate_ideal_solutions(self):
        self.positive_ideal = np.max(self.weighted_norm_matrix, axis=0)
        self.negative_ideal = np.min(self.weighted_norm_matrix, axis=0)
        print("\nPositive and Negative Ideal Solutions:")
        print(f"Positive Ideal Solution: {', '.join(f'{value:.9f}' for value in self.positive_ideal)}")
        print(f"Negative Ideal Solution: {', '.join(f'{value:.9f}' for value in self.negative_ideal)}")

    # Step 4: Calculate distances from positive and negative ideal solutions
    def calculate_distances(self):
        self.positive_distances = np.sqrt(np.sum((self.weighted_norm_matrix - self.positive_ideal) ** 2, axis=1))
        self.negative_distances = np.sqrt(np.sum((self.weighted_norm_matrix - self.negative_ideal) ** 2, axis=1))
        print("\nDistances from Positive and Negative Ideal Solutions:")
        print(tabulate(np.column_stack((self.alternatives, self.positive_distances, self.negative_distances)), headers=['Alternative', 'Positive Distance', 'Negative Distance'], floatfmt=".9f"))

    # Step 5: Calculate relative closeness to the ideal solution
    def calculate_relative_closeness(self):
        self.relative_closeness = self.negative_distances / (self.positive_distances + self.negative_distances)
        print("\nRelative Closeness to Ideal Solution:")
        print(tabulate(np.column_stack((self.alternatives, self.relative_closeness)), headers=['Alternative', 'Relative Closeness'], floatfmt=".9f"))

    # Step 6: Rank the alternatives
    def rank_alternatives(self):
        sorted_indices = np.argsort(-self.relative_closeness)
        sorted_alternatives = [self.alternatives[i] for i in sorted_indices]
        sorted_relative_closeness = [self.relative_closeness[i] for i in sorted_indices]
        ranking = np.arange(1, len(sorted_alternatives) + 1)
        results = list(zip(ranking, sorted_alternatives, sorted_relative_closeness))
        print("\nRanking of Alternatives:\n")
        print(tabulate(results, headers=['Rank', 'Alternative', 'Relative Closeness'], floatfmt=(".0f", "", ".9f")))

    # Step 7: Top-rangked Employee
    def top_ranked_employee(self):
        sorted_indices = np.argsort(-self.relative_closeness)
        sorted_alternatives = [self.alternatives[i] for i in sorted_indices]
        sorted_relative_closeness = [self.relative_closeness[i] for i in sorted_indices]
        ranking = np.arange(1, len(sorted_alternatives) + 1)
        results = list(zip(ranking, sorted_alternatives, sorted_relative_closeness))
        top_employee = results[0][1]
        top_relative_closeness = results[0][2]
        print(f"\nTop-ranked Employee: {top_employee} with Relative Closeness: {top_relative_closeness:.9f}")

topsis = TOPSIS(matrix, weights, alternatives)

## Data Tables

In [7]:
topsis.create_tabulate_alternatives()

Alternatives:
Alternative      Criterion 1    Criterion 2    Criterion 3    Criterion 4    Criterion 5
-------------  -------------  -------------  -------------  -------------  -------------
Employee 1       0.580931184    0.788040265    0.937359594    0.687902172    0.509728789
Employee 2       0.964821870    0.792546734    0.577127656    0.984837514    0.715551474
Employee 3       0.598267311    0.808703304    0.843499480    0.528232121    0.650259397
Employee 4       0.742444268    0.700378850    0.892362093    0.558131177    0.886015669
Employee 5       0.620645276    0.967608366    0.523953894    0.738085269    0.696835744
Employee 6       0.796903399    0.795078571    0.812602903    0.904874835    0.676307210
Employee 7       0.520717083    0.854832069    0.779039389    0.887748687    0.930064003
Employee 8       0.812679731    0.834769621    0.865657620    0.831197757    0.740256887
Employee 9       0.575143355    0.542237416    0.643247244    0.617809634    0.503451293
Employe

## Steps

### Step 1: Normalize the decision matrix

Langkah pertama dalam metode TOPSIS adalah normalisasi matriks keputusan. Normalisasi dilakukan untuk menghindari bias yang mungkin timbul karena perbedaan skala antar kriteria.

Langkah-langkah untuk normalisasi matriks keputusan adalah sebagai berikut:

1. **Normalisasi Kolom**:
   - Untuk setiap kriteria, bagi setiap elemen dalam kolom dengan jumlah kuadrat dari semua elemen dalam kolom tersebut.

   $$ r_{ij} = \frac{x_{ij}}{\sqrt{\sum_{i=1}^{m} x_{ij}^2}} $$

   Di sini, $ r_{ij} $ adalah nilai ternormalisasi dari elemen pada baris ke-$ i $ dan kolom ke-$ j $, dan $ x_{ij} $ adalah nilai asli dari elemen tersebut.

Setelah langkah ini selesai, matriks keputusan akan berada dalam bentuk ternormalisasi, yang siap untuk digunakan dalam langkah-langkah selanjutnya dari metode TOPSIS.

In [8]:
topsis.normalize_matrix()


Normalized Decision Matrix:
Alternative      Criterion 1    Criterion 2    Criterion 3    Criterion 4    Criterion 5
-------------  -------------  -------------  -------------  -------------  -------------
Employee 1       0.267510029    0.314275605    0.380612511    0.286705414    0.228726898
Employee 2       0.444285886    0.316072814    0.234341237    0.410462793    0.321084217
Employee 3       0.275493053    0.322516159    0.342500847    0.220157771    0.291786177
Employee 4       0.341884362    0.279315659    0.362341389    0.232619167    0.397575377
Employee 5       0.285797768    0.385888536    0.212750164    0.307620837    0.312686043
Employee 6       0.366961969    0.317082527    0.329955370    0.377135768    0.303474423
Employee 7       0.239782345    0.340912612    0.316326989    0.369997893    0.417340866
Employee 8       0.374226731    0.332911577    0.351498105    0.346428469    0.332170097
Employee 9       0.264844821    0.216247823    0.261188930    0.257492088    0.22

### Step 2: Apply weights to the normalized matrix

Untuk menerapkan bobot ke matriks ternormalisasi, setiap elemen ternormalisasi $ r_{ij} $ dikalikan dengan bobot yang sesuai $ w_j $ untuk kriteria tersebut. Rumus untuk langkah ini adalah:

$$
v_{ij} = r_{ij} \times w_j
$$

di mana:
- $ v_{ij} $ adalah nilai matriks ternormalisasi yang telah dibobotkan untuk elemen pada baris ke-$ i $ dan kolom ke-$ j $.
- $ r_{ij} $ adalah nilai ternormalisasi dari elemen pada baris ke-$ i $ dan kolom ke-$ j $.
- $ w_j $ adalah bobot dari kriteria ke-$ j $.

In [9]:
topsis.weighted_normalized_matrix()


Weighted Normalized Matrix:
Alternative      Criterion 1    Criterion 2    Criterion 3    Criterion 4    Criterion 5
-------------  -------------  -------------  -------------  -------------  -------------
Employee 1       0.006488967    0.105340308    0.132706681    0.006201421    0.061816316
Employee 2       0.010777004    0.105942705    0.081706846    0.008878286    0.086777040
Employee 3       0.006682611    0.108102414    0.119418436    0.004762000    0.078858877
Employee 4       0.008293059    0.093622277    0.126336160    0.005031539    0.107449737
Employee 5       0.006932571    0.129343852    0.074178771    0.006653821    0.084507329
Employee 6       0.008901364    0.106281145    0.115044253    0.008157425    0.082017773
Employee 7       0.005816379    0.114268620    0.110292499    0.008003033    0.112791608
Employee 8       0.009077585    0.111586797    0.122555474    0.007493228    0.089773139
Employee 9       0.006424318    0.072482916    0.091067726    0.005569539    0.06

### Step 3: Calculate positive and negative ideal solutions

Setelah matriks ternormalisasi dibobotkan, langkah selanjutnya adalah menghitung solusi ideal positif (A+) dan solusi ideal negatif (A-). Solusi ideal positif (A+) adalah vektor di mana setiap elemen adalah nilai maksimum dari setiap kolom, sedangkan solusi ideal negatif (A-) adalah vektor di mana setiap elemen adalah nilai minimum dari setiap kolom.

Langkah-langkah untuk menghitung A+ dan A- adalah sebagai berikut:

1. **A+ (Solusi Ideal Positif)**:
   - Untuk setiap kriteria $ j $, cari nilai maksimum $ v_{ij} $ dari semua alternatif.
   - Misalkan $ A+ $ adalah vektor yang berisi nilai maksimum untuk setiap kriteria.

   $$ A+ = (A_{1}^{+}, A_{2}^{+}, ..., A_{n}^{+}) $$

2. **A- (Solusi Ideal Negatif)**:
   - Untuk setiap kriteria $ j $, cari nilai minimum $ v_{ij} $ dari semua alternatif.
   - Misalkan $ A- $ adalah vektor yang berisi nilai minimum untuk setiap kriteria.

   $$ A- = (A_{1}^{-}, A_{2}^{-}, ..., A_{n}^{-}) $$

Dengan demikian, kita mendapatkan vektor solusi ideal positif $ A+ $ dan vektor solusi ideal negatif $ A- $ untuk digunakan dalam langkah selanjutnya dari metode TOPSIS.

In [10]:
topsis.calculate_ideal_solutions()


Positive and Negative Ideal Solutions:
Positive Ideal Solution: 0.010777004, 0.129343852, 0.132706681, 0.008878286, 0.112791608
Negative Ideal Solution: 0.005649395, 0.072482916, 0.074178771, 0.004762000, 0.061055025


### Step 4: Calculate distances from positive and negative ideal solutions

Setelah kita memiliki solusi ideal positif (A+) dan solusi ideal negatif (A-), langkah berikutnya dalam metode TOPSIS adalah menghitung jarak antara setiap alternatif dengan kedua solusi ideal tersebut. Jarak ini menggambarkan seberapa dekat atau seberapa jauh sebuah alternatif dari kedua solusi ideal tersebut.

Langkah-langkah untuk menghitung jarak antara setiap alternatif dengan solusi ideal positif (D+) dan solusi ideal negatif (D-) adalah sebagai berikut:

1. **Jarak dari Solusi Ideal Positif (D+)**:
   - Untuk setiap alternatif $ i $, hitung jarak Euclidean antara vektor alternatif dengan solusi ideal positif $ A+ $.
   - Rumus jarak Euclidean antara vektor $ v_i $ dan $ A+ $ adalah sebagai berikut:

     $$ D_{i}^{+} = \sqrt{\sum_{j=1}^{n} (v_{ij} - A_{j}^{+})^2} $$

2. **Jarak dari Solusi Ideal Negatif (D-)**:
   - Untuk setiap alternatif $ i $, hitung jarak Euclidean antara vektor alternatif dengan solusi ideal negatif $ A- $.
   - Rumus jarak Euclidean antara vektor $ v_i $ dan $ A- $ adalah sebagai berikut:

     $$ D_{i}^{-} = \sqrt{\sum_{j=1}^{n} (v_{ij} - A_{j}^{-})^2} $$

Dengan demikian, kita mendapatkan dua set jarak $ D+ $ dan $ D- $ yang akan digunakan dalam langkah selanjutnya dari metode TOPSIS.

In [11]:
topsis.calculate_distances()


Distances from Positive and Negative Ideal Solutions:
Alternative      Positive Distance    Negative Distance
-------------  -------------------  -------------------
Employee 1             0.056570341          0.067145222
Employee 2             0.061849451          0.043371507
Employee 3             0.042578331          0.060277862
Employee 4             0.036961027          0.072984907
Employee 5             0.065155542          0.061549997
Employee 6             0.042366454          0.057217558
Employee 7             0.027477847          0.075745974
Employee 8             0.030870963          0.068653923
Employee 9             0.087598723          0.016925999
Employee 10            0.050238508          0.051972457


### Step 5: Calculate relative closeness to the ideal solution

Setelah menghitung jarak dari setiap alternatif ke solusi ideal positif (D+) dan solusi ideal negatif (D-), langkah terakhir dalam metode TOPSIS adalah menghitung kedekatan relatif setiap alternatif dengan solusi ideal. Kedekatan relatif ini memberikan informasi tentang seberapa baik sebuah alternatif dibandingkan dengan alternatif lainnya.

Langkah-langkah untuk menghitung kedekatan relatif ($ C_i $) untuk setiap alternatif ($ i $) adalah sebagai berikut:

1. **Kedekatan Relatif**:
   - Untuk setiap alternatif $ i $, hitung kedekatan relatif ($ C_i $) menggunakan rumus berikut:

     $$ C_{i} = \frac{D_{i}^{-}}{D_{i}^{+} + D_{i}^{-}} $$

   Di sini, $ D_{i}^{+} $ adalah jarak dari alternatif $ i $ ke solusi ideal positif, dan $ D_{i}^{-} $ adalah jarak dari alternatif $ i $ ke solusi ideal negatif.

Dengan demikian, kita mendapatkan nilai kedekatan relatif ($ C_i $) untuk setiap alternatif $ i $, yang menggambarkan seberapa baik alternatif tersebut dalam kaitannya dengan solusi ideal. Semakin besar nilai $ C_i $, semakin baik kedekatan relatif alternatif tersebut dengan solusi ideal.

In [12]:
topsis.calculate_relative_closeness()


Relative Closeness to Ideal Solution:
Alternative      Relative Closeness
-------------  --------------------
Employee 1              0.542738688
Employee 2              0.412194564
Employee 3              0.586040183
Employee 4              0.663825433
Employee 5              0.485771949
Employee 6              0.574565701
Employee 7              0.733803239
Employee 8              0.689816645
Employee 9              0.161932973
Employee 10             0.508482209


### Step 6: Rank the alternatives

In [13]:
topsis.rank_alternatives()


Ranking of Alternatives:

  Rank  Alternative      Relative Closeness
------  -------------  --------------------
     1  Employee 7              0.733803239
     2  Employee 8              0.689816645
     3  Employee 4              0.663825433
     4  Employee 3              0.586040183
     5  Employee 6              0.574565701
     6  Employee 1              0.542738688
     7  Employee 10             0.508482209
     8  Employee 5              0.485771949
     9  Employee 2              0.412194564
    10  Employee 9              0.161932973


### Step 7. Top Ranked Employee

In [14]:
topsis.top_ranked_employee()


Top-ranked Employee: Employee 7 with Relative Closeness: 0.733803239
